In [5]:
#%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import join as opj
from numpy import median, rint

from nipype.interfaces import freesurfer as fs
from nipype.interfaces import spm
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/Users/myelin/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = '/Users/myelin/Documents/asl_practice/subs/freesurfer'
fs.FSCommand.set_default_subjects_dir(fs_dir)

#other study-specific variables
subjects_dir = '/Users/myelin/Documents/asl_practice/subs'
subjects_list = ['ko']
raw_dir = 'raw'
output_dir = 'proc'

#pre-proc variables
numslices = 38
TR = 4.844
asl_slice_order = range(1,numslices+1)
middle_slice = np.around(median(asl_slice_order), decimals=-1)
middle_slice = middle_slice.astype(int)

In [10]:
# preprocflow nodes

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subjid']),
                  name="infosource")
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'anat': subjects_dir + '/{subjid}/anat.nii.gz',
              'func': subjects_dir + '/{subjid}/cbf.nii.gz'}
selectfiles = Node(SelectFiles(templates), name="selectfiles")

#gunzip
gunzip = Node(Gunzip(), name="gunzip")

#slicetiming correction - need to ask Bob
slice_timing = Node(spm.SliceTiming(num_slices=numslices,
                               time_repetition=TR,
                               time_acquisition=TR-TR/numslices,
                               slice_order=asl_slice_order,
                               ref_slice=middle_slice),
                   name="slice_timing")

#smoothing
smooth = Node(spm.Smooth(), name="smooth")
smooth.inputs.fwhm = 4

#spatial normalization
norm = Node(spm.Normalize(), name="norm")

# Datasink
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = subjects_dir
datasink.inputs.container = output_dir

In [16]:
#establish asl preprocessing workflow that can be run on one subject at a time
preprocflow = Workflow(name="preprocflow")

preprocflow.base_dir = opj(subjects_dir)
preprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, gunzip, [('func', 'in_file')]),
                     (gunzip, slice_timing, [('out_file','in_files')]),
                     (slice_timing, datasink, [('timecorrected_files', 'slice_timing')]),
                     (slice_timing, norm,[('timecorrected_files','source')]),
                     (norm, datasink, [('normalized_files', 'norm')]),
                    ])
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 1})

170220-18:25:22,346 workflow INFO:
	 Creating detailed dot file: /Users/myelin/Documents/asl_practice/subs/preprocflow/graph_detailed.dot
170220-18:25:22,461 workflow INFO:
	 Creating dot file: /Users/myelin/Documents/asl_practice/subs/preprocflow/graph.dot
170220-18:25:22,561 workflow INFO:
	 ['check', 'execution', 'logging']
170220-18:25:22,582 workflow INFO:
	 Running in parallel.
170220-18:25:22,586 workflow INFO:
	 Executing: selectfiles.a0 ID: 0
170220-18:25:22,593 workflow INFO:
	 Executing node selectfiles.a0 in dir: /Users/myelin/Documents/asl_practice/subs/preprocflow/_subjid_ko/selectfiles
170220-18:25:22,631 workflow ERROR:
	 ['Node selectfiles.a0 failed to run on host myelin.stanford.edu.']
170220-18:25:22,633 workflow INFO:
	 Saving crash info to /Users/myelin/Documents/asl_practice/pcasl_processing/crash-20170220-182522-myelin-selectfiles.a0-1d06f8e5-bc2a-45b5-932f-3035bc60d314.pklz
170220-18:25:22,635 workflow INFO:
	 Traceback (most recent call last):
  File "/usr/loca

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
#establish processing workflow for analysis of multiple subjects
procflow = Workflow(name="procflow")


In [ ]:
#proc nodes


In [ ]:
modelflow = Workflow(name="modelflow")